In [6]:
#WEC-GRID Development Block

import os,sys
from pickle import HIGHEST_PROTOCOL

# Path stuff
sys.path.append(r"C:\Program Files\PTI\PSSE35\35.3\PSSPY37")
sys.path.append(r"C:\Program Files\PTI\PSSE35\35.3\PSSBIN")
sys.path.append(r"C:\Program Files\PTI\PSSE35\35.3\PSSLIB")
sys.path.append(r"C:\Program Files\PTI\PSSE35\35.3\EXAMPLE")
os.environ['PATH'] = (r"C:\Program Files\PTI\PSSE35\35.3\PSSPY37;" 
  + r"C:\Program Files\PTI\PSSE35\35.3\PSSBIN;" 
  + r"C:\Program Files\PTI\PSSE35\35.3\EXAMPLE;" + os.environ['PATH'])

# imports all the GOOOOOOD stuff we need for our program / automation stuff 
import glob
from pathlib import Path
import pandas as pd
import psse35
psse35.set_minor(3)
import math
import numpy as np
import matplotlib.pyplot as plt

import psspy
psspy.psseinit(50)
_i = psspy.getdefaultint()
_f = psspy.getdefaultreal()
_s = psspy.getdefaultchar()

class Wec_grid:
    def __init__(self, case, solver, wec_bus):

        # initalized variables and files
        self.case_file = case
        self.lst_param = ['BASE', 'PU', 'ANGLED', 'P', 'Q']
        self.dataframe = pd.DataFrame()
        self.wecBus_num = wec_bus
        self.history = {}
        self._i = psspy.getdefaultint()
        self._f = psspy.getdefaultreal()
        self._s = psspy.getdefaultchar()

        # initialization functions
        psspy.read(1, case)
        self.run_powerflow(solver)

        # program variables
        self.swingBus = self.dataframe.loc[self.dataframe['Bus'] == 'BUS 1']
        self.swingBus.insert(0, 'time', None)
        self.wecBus = self.dataframe.loc[self.dataframe['Bus'] == 'BUS {}'.format(str(self.wecBus_num))]
        self.wecBus.insert(0, 'time', None)

        self.history['Start'] = self.dataframe

    def clear():
        # initalized variables and files
        self.case_file = case
        self.lst_param = ['BASE', 'PU', 'ANGLED', 'P', 'Q']
        self.dataframe = pd.DataFrame()
        self.wecBus_num = wec_bus
        self.history = {}

        # initialization functions
        psspy.read(1, case)
        self.run_powerflow(solver)

        # program variables
        self.swingBus = self.dataframe.loc[self.dataframe['Bus'] == 'BUS 1']
        self.swingBus.insert(0, 'time', None)
        self.wecBus = self.dataframe.loc[self.dataframe['Bus'] == 'BUS {}'.format(str(self.wecBus_num))]
        self.wecBus.insert(0, 'time', None)

        self.history['Start'] = self.dataframe

    def run_powerflow(self,solver):
        if solver == 'fnsl':
            psspy.fnsl()
            self.get_values()
        if solver == 'GS':
            psspy.solv()
            self.get_values()
        if solver == 'DC':
            #psspy.dclf_2(1, 1, [1,2,12,1,0,1],[0,0,0], '1')
            psspy.dclf()
            self.get_values()
        else:
            print("error in run_pf")

    def get_values(self):
        """
        Descrtiption: 
        input: List of Parameters such as BASE, PU, KV, ANGLED 
        output: Dataframe of the selected parameters for each bus
        """
        lst = self.lst_param
        temp_dict = {}
        for i in range(len(lst)):
            #print('here {}'.format(i))
            if lst[i] != "P" and lst[i] != "Q":
                ierr, bus_voltages = psspy.abusreal(-1, string=lst[i])
                bus_add = {}
                for j in range(len(bus_voltages[0])):
                    bus_add['BUS {}'.format(j+1)] = bus_voltages[0][j]
                    temp_dict[lst[i]] = bus_voltages[0]
                temp_dict[lst[i]] = bus_add
                

        self.dataframe = pd.DataFrame.from_dict(temp_dict) 
        self.dataframe = self.dataframe.reset_index()
        self.dataframe = self.dataframe.rename(columns={'index':"Bus"})
        self.dataframe['Type'] = psspy.abusint(-1, string="TYPE")[1][0]
        self.dataframe.insert(0, "BUS_ID", range(1, 1 + len(self.dataframe)))
        self.addGeninfo()
        self.addLoadinfo()
        
        if "P" in lst:
            self.dataframe['P'] = 0 # initalize P column
            self.get_p_or_q('P')

        if "Q" in lst:
            self.dataframe['Q'] = 0 # initalize Q column
            self.get_p_or_q('Q')       
            
    def get_p_or_q(self, letter):
        """
        Descrtiption: retre P (activate) Q (reactive) Voltage (in PU) and Voltage Angle for each Bus in the current loaded case
        input:
        output:
        """      
        gen = self.dataframe['{} Gen'.format(letter)]
        load = self.dataframe['{} Load'.format(letter)]
        temp = []
        for i in range(len(gen)):
            if (not np.isnan(gen[i])) and (not np.isnan(load[i])):
                temp.append(gen[i] - load[i])
            elif np.isnan(gen[i]) and np.isnan(load[i]):
                temp.append(None)
            else:
                if np.isnan(gen[i]):
                    temp.append(-load[i])
                else:
                    temp.append(gen[i])
        self.dataframe['{}'.format(letter)] = temp
            
            
    def busNum(self):
        """
        Descrtiption: Returns the number of Buses in the currently loaded case
        input: None 
        output: Number of Buses
        """
        psspy.bsys(0,0,[0.0,0.0],1,[1],0,[],0,[],0,[])
        ierr,all_bus = psspy.abusint(0,1,['number'])
        return all_bus[0]
    
    def dc_injection(self, ibus, p, pf_solver, time):
        ierr = psspy.machine_chng_3(self.wecBus_num, "1", [self._i,self._i,self._i,self._i,self._i,self._i,self._i], [p, 0, self._f, self._f, self._f,self._f,self._f,self._f,self._f,self._f,self._f,self._f,self._f,self._f,self._f,self._f,self._f])
        if ierr > 0:
            print("Failed | machine_chng_3 code = {}".format(ierr))
        #self.run_powerflow('DC')
        
        temp = pd.DataFrame(self.dataframe.loc[self.dataframe['Bus'] == 'BUS 1'])
        temp.insert(0, 'time', time)
        self.swingBus = self.swingBus.append(temp)
        temp = pd.DataFrame(self.dataframe.loc[self.dataframe['Bus'] == 'BUS {}'.format(str(self.wecBus_num))])
        temp.insert(0,'time', time)
        self.wecBus = self.wecBus.append(temp)
        self.history[time] = self.dataframe
        
        
    def ac_injection(self, ibus, p, v, pf_solver, time):
        ierr = psspy.machine_chng_3(ibus, "1", [], [p])
        if ierr > 0:
            print("Failed | machine_chng_3 code = {}".format(ierr))
        ierr = psspy.bus_chng_4(ibus, 0, [],[self._f, v])
        if ierr > 0:
            print("Failed | bus_chng_4 code = {}".format(ierr))
        self.run_powerflow(pf_solver)
        temp = pd.DataFrame(self.dataframe.loc[self.dataframe['Bus'] == 'BUS 1'])
        temp.insert(0, 'time', time)
        self.swingBus = self.swingBus.append(temp)
        temp = pd.DataFrame(self.dataframe.loc[self.dataframe['Bus'] == 'BUS {}'.format(str(self.wecBus_num))])
        temp.insert(0,'time', time)
        self.wecBus = self.wecBus.append(temp)
        self.history[time] = self.dataframe

    def plotSwingBus(self, letter):
        plt.plot(self.swingBus.time, self.swingBus[letter], marker="o", markersize=5, markerfacecolor="green")
        plt.xlabel("Time (sec)")
        plt.ylabel("{} in MW".format(letter))
        plt.title("Swing bus")
        plt.show()
    
    def plotWecBus(self, letter):
        plt.plot(self.wecBus.time, self.wecBus[letter], marker="x", markersize=5, markerfacecolor="red")
        plt.xlabel("Time (sec)")
        plt.ylabel("{} in MW".format(letter))
        plt.title("Wec bus")
        plt.show()
        
    def addGeninfo(self):
        buses = []
        for string in psspy.amachchar(-1,1,'NAME')[1][0]:
            buses.append(self.findBusNum(string))
        temp = psspy.amachcplx(-1,1,'PQGEN')
        pointer = 0 
        pointer_1 = 0
        p_gen_df_list = []
        q_gen_df_list = []

        for index, row in self.dataframe.iterrows():
            if row['BUS_ID'] == buses[pointer]:
                p_gen_df_list.append(temp[1][0][pointer].real)
                q_gen_df_list.append(temp[1][0][pointer].imag)
                pointer +=1
                if pointer >= len(buses):
                    pointer = 0
            else:
                p_gen_df_list.append(None)
                q_gen_df_list.append(None)
        self.dataframe['P Gen'] = p_gen_df_list
        self.dataframe['Q Gen'] = q_gen_df_list

    def addLoadinfo(self):
        buses = []
        for string in psspy.aloadchar(-1,1,'NAME')[1][0]:
            buses.append(self.findBusNum(string))

        temp = psspy.aloadcplx(-1, 1, "MVAACT")
        pointer = 0 
        pointer_1 = 0
        p_load_df_list = []
        q_load_df_list = []

        for index, row in self.dataframe.iterrows():
            if row['BUS_ID'] == buses[pointer]:

                p_load_df_list.append(temp[1][0][pointer].real)
                q_load_df_list.append(temp[1][0][pointer].imag)
                pointer +=1
            else:
                p_load_df_list.append(None)
                q_load_df_list.append(None)
        self.dataframe['P Load'] = p_load_df_list
        self.dataframe['Q Load'] = q_load_df_list
        
    def findBusNum(self,string_bus_name):
        temp_string = ''
        for i in string_bus_name:
            if i.isdigit():
                temp_string += i
        return int(temp_string)

0

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [9]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
plt.rcParams["figure.figsize"] = (15,10)

In [11]:
case14 = r"../input_files/case14.raw"
w2g_obj = Wec_grid(case14,'AC', 3)
w2g_obj.dataframe


 Reading IC, SBASE, REV, XFRRAT, NXFRAT, BASFRQ...

 Converting RAW data from PSS(R)E 33 to PSS(R)E 34 format
 Starting processing of Case identification data records
 Finished processing of 3 records for Case identification data
 Starting processing of Bus data records
 Finished processing of 14 records for Bus data
 Starting processing of Load data records
 Finished processing of 11 records for Load data
 Starting processing of Fixed shunt data records
 Finished processing of 1 records for Fixed shunt data
 Starting processing of Generator data records
 Finished processing of 5 records for Generator data
 Starting processing of Branch data records
 Finished processing of 17 records for Branch data
 Finished processing of 0 records for System switching device data
 Starting processing of Transformer data records
 Finished processing of 12 records for 3 Transformer data
 Starting processing of Area data records
 Finished processing of 0 records for Area data
 Starting processing of Tw

KeyError: 'Bus'

In [12]:
ierr = psspy.machine_chng_4(3,"1",[],[psspy._i,0.0],"")
w2g_obj.get_values()


 Power flow data changed for machine "1" at bus 3 [BUS 3             ]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
    0.00000        0.100000E+09  PG
    23.4000         0.00000      QG


In [71]:
w2g_obj.dataframe

,BUS_ID,Bus,BASE,PU,ANGLED,Type,P Gen,Q Gen,P Load,Q Load,P,Q
0,1,BUS 1,0.0,1.060,0.000000,3,2.324000e+02,-16.900000,NaN,NaN,2.324000e+02,-16.900000
1,2,BUS 2,0.0,1.045,-4.980000,2,4.000000e+01,42.400002,21.700001,12.700001,1.830000e+01,29.700001
2,3,BUS 3,0.0,1.010,-12.719999,2,1.000000e+08,0.000000,94.199997,19.000000,9.999991e+07,-19.000000
3,4,BUS 4,0.0,1.019,-10.330000,1,NaN,NaN,47.799999,-3.900000,-4.780000e+01,3.900000
4,5,BUS 5,0.0,1.020,-8.780000,1,NaN,NaN,7.600000,1.600000,-7.600000e+00,-1.600000
5,6,BUS 6,0.0,1.070,-14.220000,2,0.000000e+00,12.200000,11.200000,7.500000,-1.120000e+01,4.699999
6,7,BUS 7,0.0,1.062,-13.370001,1,NaN,NaN,NaN,NaN,NaN,NaN
7,8,BUS 8,0.0,1.090,-13.360000,2,0.000000e+00,17.400000,NaN,NaN,0.000000e+00,17.400000
8,9,BUS 9,0.0,1.056,-14.939999,1,NaN,NaN,29.499998,16.600000,-2.950000e+01,-16.600000
9,10,BUS 10,0.0,1.051,-15.100001,1,NaN,NaN,9.000000,5.800000,-9.000000e+00,-5.800000


In [ ]:
case_file = '../input_files/case14.raw'

injection_file = ('../input_files/WECgen_data.csv')
injection = pd.read_csv(injection_file)

for i in range(len(injection)):
    print("Time: {}".format(injection.iloc[i].time))
    print("P setpoint: {}".format(injection.pg.iloc[i]))

    w2g_obj.dc_injection(3, injection.pg.iloc[i], 'DC', injection.iloc[i].time)
    w2g_obj.dataframe
    
    print("========================")

In [ ]:
psspy.dclf(0, 1, [1,2,1,1],[0,0,0], '1')

In [ ]:
w2g_obj.plotSwingBus("P")

In [ ]:
w2g_obj.plotWecBus('P')

In [ ]:
w2g_obj.plotWecBus('Q')

In [ ]:
for key, data in w2g_obj.history.items():
    data.loc[data['Bus'] == 'BUS 3']['P']
    break

In [ ]:
case_file = '../input_files/case14.raw' # update this with a path to an example case
psspy.read(1, case_file)
psspy.dclf_2(1, 1, [1,1,1,2,1,1],[0,0,0], '1')


In [51]:
injection_file = ('../input_files/WECgen_data.csv')
injection = pd.read_csv(injection_file)

In [52]:
injection

,time,ibus,machid,pg,qg,qt,qb,vs,ireg,nreg,...,xt,gtap,stat,rmpct,pt,pb,o1,f1,wmod,wpf
0,0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.5
1,300,1.0,1.0,0.019870,0.0,0.05,-0.05,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.019870,0.0,NaN,NaN,2.0,0.5
2,600,1.0,1.0,0.016054,0.0,0.05,-0.05,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.016054,0.0,NaN,NaN,2.0,0.5
3,900,1.0,1.0,0.020990,0.0,0.05,-0.05,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.020990,0.0,NaN,NaN,2.0,0.5
4,1200,1.0,1.0,0.016094,0.0,0.05,-0.05,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.016094,0.0,NaN,NaN,2.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,20400,1.0,1.0,0.021365,0.0,0.05,-0.05,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.021365,0.0,NaN,NaN,2.0,0.5
69,20700,1.0,1.0,0.017197,0.0,0.05,-0.05,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.017197,0.0,NaN,NaN,2.0,0.5
70,21000,1.0,1.0,0.022581,0.0,0.05,-0.05,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.022581,0.0,NaN,NaN,2.0,0.5
71,21300,1.0,1.0,0.020330,0.0,0.05,-0.05,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.020330,0.0,NaN,NaN,2.0,0.5


In [64]:
psspy.machine_chng_4(3,"1",[],[ 0.03,0.0],"")


 Power flow data changed for machine "1" at bus 3 [BUS 3             ]:
 X--ORIGINAL--X  X-NEW VALUE--X  DATA ITEM
   0.200000E-01    0.300000E-01  PG


0

In [59]:
psspy.machine_chng_4??